In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import sklearn
import cv2
import scipy
import keras
import glob
import os
import cv2

In [ ]:
print("CV2", cv2.__version__)
print("Tensorflow",tf.__version__)
print("Numpy",np.__version__)
print("sklearn",sklearn.__version__)
print("scipy",scipy.__version__)
print("keras",keras.__version__)

In [ ]:
file_list = glob.glob("./modified_data/*")
print(len(file_list), "number of files")

In [ ]:
# Reading our training CSV file
with open("./train_data.csv", "r") as f:
    lines = f.readlines()
    
dataset = []

for line in lines[1:]:
    
    sline = line.strip().split(",")
    dataset.append({
        "file": sline[0],
        "person_id": int(sline[1]),
        "series": int(sline[2]),
        "tilt": int(sline[3]),
        "pan": int(sline[4])
    })

In [ ]:
# Displaying tilt and pan angles
tilts = [item["tilt"] for item in dataset]
pans = [item["pan"] for item in dataset]

all_tilts = np.unique(tilts)
all_pans = np.unique(pans)

print ("Tilt angles : ", all_tilts)
print ("Pan angles : ", all_pans)

# Group Classes

In [ ]:
# Grouping tilt and pan angle
grouped_classes = {}
class_groups = {}
class_counter = 0

for tilt in all_tilts:
    grouped_classes[tilt] = {}
    for pan in all_pans:
        grouped_classes[tilt][pan] = class_counter
        class_groups[class_counter] = {"tilt": tilt, "pan": pan}
        class_counter+=1
        
class_groups

In [ ]:
# Storing image path and their tilt and pan angle in lists respectively
x_data = []
y_data = []

for item in dataset:
    x_data.append(item["file"])
    y_data.append(grouped_classes[item["tilt"]][item["pan"]])

x_data, y_data

In [ ]:
# Displaying no of classes after grouping our tilt and pan angles
n_classes = len(np.unique(y_data))
print("Grouped Classes : ",n_classes)

# Grouped Classification Model

In [ ]:
# Splitting our data for training and valdiation
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [ ]:
# Displaying our training and validation data count
print ('x_train : ', len(x_train))
print ('y_train : ', len(y_train))
print ('x_test : ', len(x_test))
print ('y_test : ', len(y_test))

### Data PreProcessing and Creating Data Directories

In [ ]:
# Resizing our training images
images_base_dir = "modified_data/"
save_dir = "grouped_data/"

os.makedirs(save_dir, exist_ok=True)

for imgName, imgClass in zip(x_train, y_train):
    dataimg = cv2.imread(images_base_dir + imgName)
    dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
    dataimg = cv2.resize(dataimg, (224, 224))
    
    os.makedirs( save_dir + "train/%d/" % imgClass, exist_ok=True)
    cv2.imwrite(save_dir + "train/%d/%s" % (imgClass, imgName), dataimg)


In [ ]:
# Resizing our validation images
for imgName, imgClass in zip(x_test, y_test):
    dataimg = cv2.imread(images_base_dir + imgName)
    dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
    dataimg = cv2.resize(dataimg, (224, 224))
    
    os.makedirs( save_dir + "test/%d/" % imgClass, exist_ok=True)
    cv2.imwrite(save_dir + "test/%d/%s" % (imgClass, imgName), dataimg)

In [ ]:
# Loading mobilenetv2 model
base_model= tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False)

In [ ]:
classifciation_model = base_model.output
classifciation_model = tf.keras.layers.GlobalAveragePooling2D()(classifciation_model)
classifciation_model = tf.keras.layers.Dense(1024, activation='relu')(classifciation_model)
classifciation_model = tf.keras.layers.Dropout(0.5)(classifciation_model)
classifciation_model = tf.keras.layers.Dense(512, activation='relu')(classifciation_model)
classifciation_model = tf.keras.layers.Dropout(0.5)(classifciation_model)
classifciation_model = tf.keras.layers.Dense(256, activation='relu')(classifciation_model) 
classification_out = tf.keras.layers.Dense(93, activation='softmax')(classifciation_model) 

In [ ]:
groupedClassificationModel = tf.keras.Model(inputs=base_model.input,outputs=classification_out)

In [ ]:
# Generate batches of tensor image data for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
train_generator= train_datagen.flow_from_directory('./grouped_data/train/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
# Generate batches of tensor image data for validation
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
test_generator= test_datagen.flow_from_directory('./grouped_data/test/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
# Compiling and fiting our model
groupedClassificationModel.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
sst=train_generator.n//train_generator.batch_size
groupedClassificationModel.fit_generator(generator=train_generator, steps_per_epoch=sst, epochs=10)

In [ ]:
# Running 30 more epochs
groupedClassificationModel.fit_generator(generator=train_generator, steps_per_epoch=sst, epochs=30)

In [ ]:
# Running 30 more epochs
groupedClassificationModel.fit_generator(generator=train_generator, steps_per_epoch=sst, epochs=30)

In [ ]:
# Saving our grouped classification model to a groupedClassificationModel directory
model_dir = "groupedClassificationModel"
os.makedirs(model_dir, exist_ok=True)
groupedClassificationModel.save("%s/model.h5" % model_dir)

In [ ]:
# Saving our mapping interger mapping to classes in a np file
indices_to_class = {}

for k, val in train_generator.class_indices.items():
    indices_to_class[val] = k
    
np.save("%s/mapping.np" % model_dir, indices_to_class)

In [ ]:
# Predicting our trained model validation accuracy for both tilt and pan angle
tilt_accuracy = 0
pan_accuracy = 0

for x, y in zip(x_test, y_test):
        
    predImg = cv2.imread(save_dir + "test/%d/%s" % (y, x))
    predImg = tf.keras.applications.mobilenet_v2.preprocess_input(predImg)
    pred = groupedClassificationModel.predict(np.array([predImg]))
    
    predictedClass = class_groups[int(indices_to_class[np.argmax(pred)])]
    actualClass = class_groups[y]
    
    if predictedClass["tilt"] == actualClass["tilt"]:
        tilt_accuracy+=1
    if predictedClass["pan"] == actualClass["pan"]:
        pan_accuracy+=1
        
        
print("Tilt accuracy", tilt_accuracy / len(y_test) * 100)
print("Pan accuracy", pan_accuracy / len(y_test) * 100)

# Two classification models

In [ ]:
# Storing image path and their tilt and pan angle in lists respectively
x_data = []
y_data = []

for item in dataset:
    x_data.append(item["file"])
    y_data.append({"tilt": item["tilt"], "pan" : item["pan"]})

y_data

In [ ]:
# Splitting our data for training and valdiation
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [ ]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

### Data PreProcessing and Creating Data Directories

In [ ]:
# Resizing our training images and validation images
images_base_dir = "modified_data/"
save_dir = "bi_classification_data/"

os.makedirs(save_dir, exist_ok=True)
os.makedirs(save_dir + "/tilt/", exist_ok=True)
os.makedirs(save_dir + "/pan/", exist_ok=True)

for imgName, imgClasses in zip(x_train, y_train):
    
    dataimg = cv2.imread(images_base_dir + imgName)
    dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
    dataimg = cv2.resize(dataimg, (224, 224))
    
    os.makedirs(save_dir + "tilt/train/%d/" % imgClasses["tilt"], exist_ok=True)
    cv2.imwrite(save_dir + "tilt/train/%d/%s" % (imgClasses["tilt"], imgName), dataimg)
    
    os.makedirs(save_dir + "pan/train/%d/" % imgClasses["pan"], exist_ok=True)
    cv2.imwrite(save_dir + "pan/train/%d/%s" % (imgClasses["pan"], imgName), dataimg)
    

for imgName, imgClasses in zip(x_test, y_test):
    
    dataimg = cv2.imread(images_base_dir + imgName)
    dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
    dataimg = cv2.resize(dataimg, (224, 224))
    
    os.makedirs(save_dir + "tilt/test/%d/" % imgClasses["tilt"], exist_ok=True)
    cv2.imwrite(save_dir + "tilt/test/%d/%s" % (imgClasses["tilt"], imgName), dataimg)
    
    os.makedirs(save_dir + "pan/test/%d/" % imgClasses["pan"], exist_ok=True)
    cv2.imwrite(save_dir + "pan/test/%d/%s" % (imgClasses["pan"], imgName), dataimg)

In [ ]:
# Loading mobilenetv2 model
base_model= tf.keras.applications.MobileNetV2(weights='imagenet',include_top=False)
tilt_classifciation_model = base_model.output
tilt_classifciation_model = tf.keras.layers.GlobalAveragePooling2D()(tilt_classifciation_model)
tilt_classifciation_model = tf.keras.layers.Dense(256, activation='relu')(tilt_classifciation_model)
tilt_classifciation_model = tf.keras.layers.Dropout(0.5)(tilt_classifciation_model)
tilt_classifciation_model = tf.keras.layers.Dense(128, activation='relu')(tilt_classifciation_model)
tilt_classifciation_model = tf.keras.layers.Dropout(0.5)(tilt_classifciation_model)
tilt_classifciation_model = tf.keras.layers.Dense(64, activation='relu')(tilt_classifciation_model) 
tilt_classification_out = tf.keras.layers.Dense(len(all_tilts), activation='softmax')(tilt_classifciation_model) 

In [ ]:
tiltClassificationModel = tf.keras.Model(inputs=base_model.input,outputs=tilt_classification_out)

In [ ]:
# Generate batches of tensor image data for training
tilt_train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
tilt_train_generator= tilt_train_datagen.flow_from_directory('./bi_classification_data/tilt/train/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
# Compiling and fiting our model
tiltClassificationModel.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
sst=tilt_train_generator.n//tilt_train_generator.batch_size
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=1)

In [ ]:
# Running 5 more epochs
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=5)

In [ ]:
# Running 10 more epochs
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=10)

In [ ]:
# Running 10 more epochs
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=10)

In [ ]:
# Running 5 more epochs
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=5)

In [ ]:
# Running 5 more epochs
tiltClassificationModel.fit_generator(generator=tilt_train_generator, steps_per_epoch=sst, epochs=5)

In [ ]:
# Saving our tilt classification model to a twoClassificationModels directory
model_dir = "twoClassificationModels"
os.makedirs(model_dir, exist_ok=True)
tiltClassificationModel.save("%s/tilt_model.h5" % model_dir)

In [ ]:
# Loading mobilenetv2 model
base_model= tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)
pan_classifciation_model = base_model.output
pan_classifciation_model = tf.keras.layers.GlobalAveragePooling2D()(pan_classifciation_model)
pan_classifciation_model = tf.keras.layers.Dense(256, activation='relu')(pan_classifciation_model)
pan_classifciation_model = tf.keras.layers.Dropout(0.5)(pan_classifciation_model)
pan_classifciation_model = tf.keras.layers.Dense(128, activation='relu')(pan_classifciation_model)
pan_classifciation_model = tf.keras.layers.Dropout(0.5)(pan_classifciation_model)
pan_classifciation_model = tf.keras.layers.Dense(64, activation='relu')(pan_classifciation_model) 
pan_classification_out = tf.keras.layers.Dense(len(all_pans), activation='softmax')(pan_classifciation_model) 

In [ ]:
panClassificationModel = tf.keras.Model(inputs=base_model.input,outputs=pan_classification_out)

In [ ]:
# Generate batches of tensor image data for training
pan_train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
pan_train_generator= pan_train_datagen.flow_from_directory('./bi_classification_data/pan/train/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
# Compiling and fiting our model
panClassificationModel.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
sst=pan_train_generator.n//pan_train_generator.batch_size
panClassificationModel.fit_generator(generator=pan_train_generator, steps_per_epoch=sst, epochs=30)

In [ ]:
# Running 5 more epochs
panClassificationModel.fit_generator(generator=pan_train_generator, steps_per_epoch=sst, epochs=5)

In [ ]:
# Running 5 more epochs
panClassificationModel.fit_generator(generator=pan_train_generator, steps_per_epoch=sst, epochs=5)

In [ ]:
# Saving our pan classification model to a twoClassificationModels directory
panClassificationModel.save("%s/pan_model.h5" % model_dir)

## Evaluation of Two classification models

In [ ]:
# Generate batches of tensor image data for validation
tilt_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
tilt_test_generator= tilt_test_datagen.flow_from_directory('./bi_classification_data/tilt/test/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
print("Tilt Classifier accuracy on evaluation dataset:",tiltClassificationModel.evaluate_generator(tilt_test_generator)[1] * 100)

In [ ]:
# Generate batches of tensor image data for validation
pan_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
pan_test_generator= pan_test_datagen.flow_from_directory('./bi_classification_data/pan/test/', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
print("Pan Classifier accuracy on evaluation dataset:", panClassificationModel.evaluate_generator(pan_test_generator)[1] * 100)

# Testing Data

In [ ]:
# Reading test CSV file
with open("./test_data.csv", "r") as f:
    lines = f.readlines()
    
test_dataset = []

for line in lines[1:]:
    sline = line.strip().split(",")
    test_dataset.append(sline[0])
    
len(test_dataset)

In [ ]:
# Reading a np file with mapping of interger to our respective tilt and pan angle
tilt_indices_to_class = {}
for k, val in tilt_train_generator.class_indices.items():
    tilt_indices_to_class[val] = k
    
pan_indices_to_class = {}
for k, val in pan_train_generator.class_indices.items():
    pan_indices_to_class[val] = k
    
tilt_indices_to_class, pan_indices_to_class

In [ ]:
# Making Prediction with our trained two classifier model on test data set for submission
images_base_dir = "modified_data/"

with open("s3802338_predictions.csv", "w") as f:
    
    f.write("filename, tilt, pan\n")
    for idx, imgName in enumerate(test_dataset):
        print(idx+1, "/", len(test_dataset))
        dataimg = cv2.imread(images_base_dir + imgName)
        dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
        dataimg = cv2.resize(dataimg, (224, 224))
        dataimg = tf.keras.applications.mobilenet_v2.preprocess_input(dataimg)

        predictedTilt = tiltClassificationModel.predict(np.array([dataimg]))
        predictedPan = panClassificationModel.predict(np.array([dataimg]))
        
        predictedTilt = int(tilt_indices_to_class[np.argmax(predictedTilt)])
        predictedPan = int(pan_indices_to_class[np.argmax(predictedPan)])
        
        f.write("%s, %d, %d\n" % (imgName, predictedTilt, predictedPan))

# Testing on unseen data

In [ ]:
from IPython.display import Image

In [ ]:
# Testing our model on unseen data
unseen = glob.glob("unseen_data/*")

In [ ]:
def test_image(img_path):
    
    dataimg = cv2.imread(img_path)
    dataimg = cv2.cvtColor(dataimg, cv2.COLOR_BGR2RGB)
    dataimg = cv2.resize(dataimg, (224, 224))
    dataimg = tf.keras.applications.mobilenet_v2.preprocess_input(dataimg)

    predictedTilt = tiltClassificationModel.predict(np.array([dataimg]))
    predictedPan = panClassificationModel.predict(np.array([dataimg]))

    predictedTilt = int(tilt_indices_to_class[np.argmax(predictedTilt)])
    predictedPan = int(pan_indices_to_class[np.argmax(predictedPan)])

    print("The tilt in the image is %d degrees" % predictedTilt)
    print("The pan in the image is %d degrees" % predictedPan)


In [ ]:
selectedIdx = 0
test_image(unseen[selectedIdx])
Image(filename=unseen[selectedIdx])

In [ ]:
selectedIdx = 1
test_image(unseen[selectedIdx])
Image(filename=unseen[selectedIdx])

In [ ]:
selectedIdx = 2
test_image(unseen[selectedIdx])
Image(filename=unseen[selectedIdx])